对测试数据进行记录

### 1. Base-model在六个领域上的基线



In [1]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')

import pandas as pd
from datasets import Dataset 
from ragas.metrics import answer_relevancy,answer_correctness,answer_correctness
from ragas import evaluate
data_path = "eval_reasult"
# gpt-4 ; gpt-3.5-turbo ; glm-3-turbo ; glm-4 ; rag

def eval_ar(q_lis:list,a_lis:list,t_lis):
    data_samples = {
        'question':q_lis,
        'answer':a_lis,
        'contexts':[[""] for i in range(len(q_lis))],
        'ground_truth':t_lis
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[answer_relevancy,answer_correctness])
    df = score.to_pandas()
    return df['answer_relevancy'].mean(),df['answer_correctness'].mean()

def process_Di(mt,data:list,_i = 1)->dict:
    # 返回模型的AC和AR
    dir ={f"AC{_i}":[],f'AR{_i}':[]}
    path = f"{data_path}\\{_i}"
    t_lis = [a['Answer'] for a in data]
    for _m in mt:
        m_data = f"{path}\eval_{_m}.csv"
        if _m == 'rag':
            m_data = "eval_reasult\\1\\new_eval_rag.csv"
        m_df = pd.read_csv(m_data).dropna(subset=['prediction'],axis=0)
        # _ac = m_df['answer_correctness'].mean()
        _ar,_ac = eval_ar(a_lis=list(m_df['prediction']),
                        q_lis=list(m_df['Question']),
                        t_lis=t_lis)
        print(f"{_m} 的平均相关性为 {_ar}")
        dir[f'AC{_i}'].append(_ac)
        print(f"{_m} 的平均正确率为 {_ac}")
        dir[f'AR{_i}'].append(_ar)
    return dir


In [ ]:
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import time
mt = ['glm-3-turbo','gpt-4','gpt-3.5-turbo','rag']#'glm-4'
d_lis = []
out_df = pd.DataFrame([],index=mt,columns=d_lis)
for i in range(1,7):
    out_df = pd.read_csv("eval_reasult2\\basellm_score.csv",index_col=0)
    if i <2:
        continue
    print(out_df.shape)
    data = DataLoader.load_json(f"_data\\testqa2\\testdata_{i}.json")
    d = process_Di(mt,_i=i,data=data)
    for _k in d.keys():
        out_df[_k] = d[_k]
    out_df.to_csv("eval_reasult2\\basellm_score.csv")
    time.sleep(10)
out_df

In [ ]:
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import time
mt = ['glm-4']#'glm-4'

row_df = pd.DataFrame([])
for i in range(1,7):
    
    if i <1:
        continue
    data = DataLoader.load_json(f"_data\\testqa2\\testdata_{i}.json")
    d = process_Di(mt,_i=i,data=data)
    for _k in d.keys():
        row_df[_k] = d[_k]
    time.sleep(10)
row_df

In [16]:
row_df.index  = mt
df = pd.concat([out_df,row_df],axis=0)
# out_df.to_csv("eval_reasult2\\basellm_score.csv",index_col=0)
new_order = [f'AC{i}' for i in range(1,7)] +[f"AR{i}" for i in range(1,7)]
df = df[new_order]

In [17]:
df.to_csv("eval_reasult2\\basellm_score.csv")

In [3]:
# 检查评测数据中是否存在因为网络问题产生的缺失
from PK_LLM_endfront.evaluation.eva_funcs import DataLoader
import pandas as pd
mt = ['gpt-4','gpt-3.5-turbo','glm-3-turbo','rag','glm-4']#'
def check(df,data):
    flag = False
    for _ in data:
        q = _['Question']
        if q not in list(df['Question']):
            print(q)
            return True
    return False


def remake_df(df:pd.DataFrame,data):
    r_df = pd.DataFrame([])
    for _ in data:
        q = _['Question']
        _df = df[df['Question']==q]
        _df = _df.iloc[-1,:]
        r_df = pd.concat([r_df,_df],axis=1)
    return r_df.T

# 检查因为未知原因出现的问答重复问题
for i in range(1,7):
    dataloader = DataLoader(f"_data\\testqa\\testdata_{i}.json")
    data = dataloader.load_json(dataloader.filepath)
    for _ in mt:
        _p = f"eval_reasult\\{i}/eval_{_}.csv"
        if _ == 'rag':
            _p = f"eval_reasult\\{i}\\new_eval_rag.csv"
        df = pd.read_csv(_p)
        if df.shape[0] != 50:
            print(f"{_} 的第{i}次数据",df.shape)
            #df.to_csv(f"eval_reasult\\{i}/eval_{_}_copy.csv")
            #r_df = remake_df(df,data)
            #print(r_df.shape)
            #r_df.to_csv(_p)
            #if check(df,data):
            #    print(f"{_} 的第{i}次数据,有缺陷")
        has_missing_values = df['prediction'].isnull().any()
        if has_missing_values:
            print(f"{_} 的第{i}次数据存在缺失 ")

In [4]:
"""
def remake_df(df:pd.DataFrame,data):
    r_df = pd.DataFrame([])
    for _ in data:
        q = _['Question']
        _df = df[df['Question']==q]
        _df = _df.iloc[-1,:]
        r_df = pd.concat([r_df,_df],axis=1)
    return r_df.T
df =pd.read_csv( f"eval_reasult\\{1}/eval_glm-3-turbo.csv")
data = dataloader.load_json(f"_data\\testqa\\testdata_{1}.json")
r = remake_df(df,data)
r.shape
"""

'\ndef remake_df(df:pd.DataFrame,data):\n    r_df = pd.DataFrame([])\n    for _ in data:\n        q = _[\'Question\']\n        _df = df[df[\'Question\']==q]\n        _df = _df.iloc[-1,:]\n        r_df = pd.concat([r_df,_df],axis=1)\n    return r_df.T\ndf =pd.read_csv( f"eval_reasult\\{1}/eval_glm-3-turbo.csv")\ndata = dataloader.load_json(f"_data\\testqa\\testdata_{1}.json")\nr = remake_df(df,data)\nr.shape\n'

In [70]:
df = pd.read_csv(f"eval_reasult\\{1}/eval_glm-4.csv")
df.shape[0]

50

In [71]:

def check(df,data):
    flag = False
    i=0
    for _ in data:
        q = _['Question']
        if q not in list(df['Question']):
            print(q)
            flag =  True
    return flag

for i in range(1,7):
    if i != 1:
        continue
    print(i)
    data = DataLoader.load_json(f"_data\\testqa\\testdata_{i}.json")
    df = pd.read_csv(f"eval_reasult\\{i}/eval_glm-4.csv")
    print(df.iloc[0,:])
    df2 = pd.read_csv(f"eval_reasult\\{i}/eval_glm-3-turbo.csv")
    #if df.shape[0] == 50:
    #    continue
    r_df = pd.DataFrame([])
    for _ in data:
        q = _['Question']
        _df = df[df['Question']==q]
        if _df.shape[0] ==0 or _df['prediction'].isnull().any():
            _df = df2[df2['Question']==q]
        _df_ = df.iloc[-1,:]
        r_df = pd.concat([r_df,_df_],axis=1)
    print(r_df.T.shape)
    df.to_csv(f"eval_reasult\\{i}/eval_glm-4_copy.csv")
    r_df.T.to_csv(f"eval_reasult\\{i}/eval_glm-4.csv")

1
Unnamed: 0.6          55.0
Unnamed: 0.5          55.0
Unnamed: 0.4          55.0
Unnamed: 0.3          55.0
Unnamed: 0.2          55.0
Unnamed: 0.1          55.0
Unnamed: 0            55.0
data                   NaN
prediction             NaN
Question               NaN
answer_similarity      NaN
answer_correctness     NaN
Name: 0, dtype: float64
(50, 12)


In [22]:
def remake_df(df:pd.DataFrame,data):
    r_df = pd.DataFrame([])
    for _ in data:
        q = _['Question']
        _df = df[df['Question']==q]
        _df = _df.iloc[-1,:]
        r_df = pd.concat([r_df,_df],axis=1)
    return r_df.T
rdf = remake_df(df2,data)

In [23]:
rdf.shape

(50, 5)

In [19]:
df = pd.read_csv("eval_reasult\\basellm_score.csv",index_col=0)
df

,AC1,AC2,AC3,AC4,AC5,AC6,AR1,AR2,AR3,AR4,AR5,AR6
gpt-4,0.683,0.688,0.737,0.723,0.703,0.711,0.965,0.962,0.962,0.937,0.913,0.925
gpt-3.5-turbo,0.684,0.706,0.714,0.717,0.717,0.760,0.961,0.962,0.953,0.949,0.933,0.933
glm-3-turbo,0.676,0.671,0.670,0.724,0.662,0.685,0.836,0.851,0.871,0.872,0.773,0.811
glm-4,0.702,0.714,0.675,0.667,0.663,0.717,0.953,0.921,0.924,0.897,0.863,0.905


In [21]:

for i in range(1,7):
    d = process_Di(['rag'],i)


rag 的平均正确率为 0.6334727763571913


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

rag 的平均相关性为 0.981276611393622
rag 的平均正确率为 0.5390943336555015


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

rag 的平均相关性为 0.7398016257310778
rag 的平均正确率为 0.47858869667478093


...

rag 的平均相关性为 0.5890690008951945
rag 的平均正确率为 0.42573479674141423


Evaluating:   0%|          | 0/250 [00:00<?, ?it/s]

rag 的平均相关性为 0.5009171375999143
rag 的平均正确率为 0.427610061819362


Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

rag 的平均相关性为 0.512975632121791
